In [1]:
import pandas as pd
import math
import matplotlib.pyplot as plt

In [2]:
def extract_chunk(df,idx,n_hist,n_pred,file,target,include_day):
    valid = True
    chunk = None
    num_cities = df.shape[1]-1
    if (idx+n_hist+n_pred+1)*24 > df.shape[0]:
        valid = False
        return valid, chunk
    chunk = df.iloc[idx*24:(idx+n_hist)*24,:-1].T
    columns = []
    for i in range(n_hist-1,-1,-1):
        for j in range(12,12+24):
            columns.append(file+'_'+str(i)+'_'+str(j % 24))
    chunk.columns = columns
    if include_day:
        day = df.iloc[idx*24:(idx+n_hist)*24,-1].reset_index(drop=True)
        chunk.insert(0,'day_of_year',day[0].day_of_year)
    if file == target:
        for i in range(n_pred):
            next_mean = df.iloc[(idx+n_hist+i)*24:(idx+n_hist+i+1)*24,:-1].T.mean(axis=1,skipna=False)
            chunk.insert(0,'mean_'+target+'_'+str(i+1),next_mean)
    chunk.index = range(idx*num_cities,(idx+1)*num_cities)
    return valid, chunk

In [3]:
n_pred = 14
n_hist = 1
target = 'temperature'
file_list = ['temperature','pressure','humidity','wind_direction','wind_speed', 'latitude', 'elevation', 'weather_description']
keep_list = ['Vancouver', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles',
        'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque', 'Denver',
        'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis',
        'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta',
        'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh',
        'Toronto', 'Philadelphia', 'New York', 'Montreal', 'Boston',
       'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa', 'Nahariyya',
       'Jerusalem', 'datetime']
master_df = pd.DataFrame()
for file in file_list:
    if file == target:
        include_day = True
    else:
        include_day = False
    df = pd.read_csv(file+'.csv')
    df = df.loc[:,keep_list]
    

    # changed = (df.iloc[:,1:].astype('float16') != df.iloc[:,1:]).sum().sum() - df.isna().sum().sum()
    temp = pd.to_datetime(df.datetime)
    # df = df.iloc[:,1:].astype('float16')
    # df['datetime'] = temp
    # print('Converting',file+'.csv to 16-bit fp\n\tNumber of changed values =',changed)
    ######################################
    df = df.drop('datetime',axis=1)
    df['datetime'] = temp

    gaps = 0
    for i in range(1,df.shape[0]):
        if (df.datetime[i].hour-df.datetime[i-1].hour) % 24 != 1:
            gaps = gaps + 1
    print('Checking',file+'.csv for missing values\n\t'+str(df.isna().sum().sum()),'NaNs\n\t'+str(gaps),'hour gaps')

    print('Checking',file+'.csv size\n\t'+str(df.shape[0]),'rows x',df.shape[1],'columns')
    
    valid = True
    big_chunk = pd.DataFrame()
    idx = 0;
    while valid:
        valid, chunk = extract_chunk(df,idx,n_hist,n_pred,file,target,include_day)
        if big_chunk.shape[0] == 0:
            big_chunk = chunk
        elif valid:
            big_chunk = pd.concat([big_chunk,chunk])
        idx = idx + 1
    if master_df.shape[0] == 0:
            master_df = big_chunk
    else:
        master_df = master_df.join(big_chunk)

Checking temperature.csv for missing values
	8030 NaNs
	0 hour gaps
Checking temperature.csv size
	45253 rows x 37 columns
Checking pressure.csv for missing values
	16680 NaNs
	0 hour gaps
Checking pressure.csv size
	45253 rows x 37 columns
Checking humidity.csv for missing values
	28651 NaNs
	0 hour gaps
Checking humidity.csv size
	45253 rows x 37 columns
Checking wind_direction.csv for missing values
	7975 NaNs
	0 hour gaps
Checking wind_direction.csv size
	45253 rows x 37 columns
Checking wind_speed.csv for missing values
	7993 NaNs
	0 hour gaps
Checking wind_speed.csv size
	45253 rows x 37 columns
Checking latitude.csv for missing values
	0 NaNs
	0 hour gaps
Checking latitude.csv size
	45252 rows x 37 columns
Checking elevation.csv for missing values
	0 NaNs
	0 hour gaps
Checking elevation.csv size
	45252 rows x 37 columns
Checking weather_description.csv for missing values
	7955 NaNs
	0 hour gaps
Checking weather_description.csv size
	45253 rows x 37 columns


In [4]:
master_df.shape

(67320, 207)

In [5]:
master_df.dropna().shape

(60847, 207)

In [6]:
master_df = master_df.dropna()

In [7]:
master_df.to_hdf('big_data_with_city.h5','master_df',mode='w',format='table',complevel=9)

In [8]:
master_df.head()

,mean_temperature_14,mean_temperature_13,mean_temperature_12,mean_temperature_11,mean_temperature_10,mean_temperature_9,mean_temperature_8,mean_temperature_7,mean_temperature_6,mean_temperature_5,...,weather_description_0_2,weather_description_0_3,weather_description_0_4,weather_description_0_5,weather_description_0_6,weather_description_0_7,weather_description_0_8,weather_description_0_9,weather_description_0_10,weather_description_0_11
32,301.820000,301.871458,301.850625,300.702292,299.662292,299.835625,298.917917,299.906042,301.140417,299.718333,...,broken clouds,broken clouds,broken clouds,broken clouds,broken clouds,broken clouds,broken clouds,broken clouds,broken clouds,broken clouds
37,289.209167,288.393333,287.710833,284.055417,284.982500,285.080833,285.735000,284.349583,286.960833,288.664583,...,few clouds,few clouds,sky is clear,sky is clear,sky is clear,few clouds,sky is clear,few clouds,broken clouds,few clouds
38,291.423750,289.817083,289.204583,286.772083,287.036667,288.661667,289.655833,288.217500,288.348542,288.943333,...,sky is clear,sky is clear,sky is clear,sky is clear,haze,broken clouds,broken clouds,sky is clear,sky is clear,broken clouds
39,286.851250,287.790833,287.182083,284.997500,284.630833,284.763750,287.650000,286.698750,286.762708,286.956667,...,broken clouds,broken clouds,broken clouds,broken clouds,overcast clouds,overcast clouds,overcast clouds,broken clouds,overcast clouds,scattered clouds
40,295.885833,295.260000,291.550000,289.724167,289.685417,291.326458,292.360000,291.697917,293.228958,293.704167,...,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear,sky is clear


In [9]:
for i in range(master_df.shape[1]):
    print(master_df.columns[i],master_df.iloc[0,i])

mean_temperature_14 301.82
mean_temperature_13 301.87145833333335
mean_temperature_12 301.850625
mean_temperature_11 300.70229166666667
mean_temperature_10 299.6622916666667
mean_temperature_9 299.835625
mean_temperature_8 298.91791666666666
mean_temperature_7 299.9060416666666
mean_temperature_6 301.1404166666667
mean_temperature_5 299.7183333333333
mean_temperature_4 299.908125
mean_temperature_3 300.6347916666667
mean_temperature_2 302.85645833333336
mean_temperature_1 302.7954326922916
day_of_year 275
temperature_0_12 309.1
temperature_0_13 310.58
temperature_0_14 310.495769231
temperature_0_15 310.411538462
temperature_0_16 310.327307692
temperature_0_17 310.243076923
temperature_0_18 310.158846154
temperature_0_19 310.074615385
temperature_0_20 309.990384615
temperature_0_21 309.906153846
temperature_0_22 309.821923077
temperature_0_23 309.737692308
temperature_0_0 309.653461538
temperature_0_1 309.569230769
temperature_0_2 309.485
temperature_0_3 309.400769231
temperature_0_4 30

In [10]:
df.columns

Index(['Vancouver', 'Portland', 'San Francisco', 'Seattle', 'Los Angeles',
       'San Diego', 'Las Vegas', 'Phoenix', 'Albuquerque', 'Denver',
       'San Antonio', 'Dallas', 'Houston', 'Kansas City', 'Minneapolis',
       'Saint Louis', 'Chicago', 'Nashville', 'Indianapolis', 'Atlanta',
       'Detroit', 'Jacksonville', 'Charlotte', 'Miami', 'Pittsburgh',
       'Toronto', 'Philadelphia', 'New York', 'Montreal', 'Boston',
       'Beersheba', 'Tel Aviv District', 'Eilat', 'Haifa', 'Nahariyya',
       'Jerusalem', 'datetime'],
      dtype='object')